<a href="https://colab.research.google.com/github/dansojo/Medical_CV/blob/main/efficientnet_model_training_%EB%8F%99%EB%AC%BC_%EA%B7%BC%EA%B3%A8%EA%B2%A9%EA%B3%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.optim as optim

 흑백 / 3채널 이미지 수

In [ ]:
# # 이미지 경로 설정
# root_dir = '/content/drive/MyDrive/Medical_CV/3/전처리_데이터'

# # 각 split 폴더 (train, val, test) 확인
# splits = ['train', 'val', 'test']

# single_channel_count = 0
# three_channel_count = 0

# for split in splits:
#     split_dir = os.path.join(root_dir, split)
#     files = [f for f in os.listdir(split_dir) if f.endswith('.npy')]

#     for file in files:
#         file_path = os.path.join(split_dir, file)

#         # 넘파이 파일 불러오기
#         data = np.load(file_path, allow_pickle=True).item()
#         image_np = data['image']

#         # 채널 수 확인
#         if (len(image_np.shape) == 2) or (len(image_np.shape) == 3 and image_np.shape[0] == 1):  # 흑백 이미지 (H, W)
#             single_channel_count += 1
#         elif len(image_np.shape) == 3 and image_np.shape[2] == 3:  # 3채널 이미지 (H, W, 3)
#             three_channel_count += 1

# # 결과 출력
# print(f"Single-channel (Grayscale) images: {single_channel_count}")
# print(f"Three-channel (RGB) images: {three_channel_count}")

Single-channel (Grayscale) images: 5642
Three-channel (RGB) images: 0


**Custom Dataset 클래스 정의**

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None, apply_clahe=False):
        self.root_dir = root_dir
        self.files = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith('.npy')]
        self.transform = transform
        self.apply_clahe = apply_clahe

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # 넘파이 파일 불러오기
        data = np.load(self.files[idx], allow_pickle=True).item()
        image_np, label = data['image'], data['label']

        # (1, H, W) 형식을 (3, H, W)로 변환
        if image_np.shape[0] == 1:  # 단일 채널인 경우
            image_np = np.repeat(image_np, 3, axis=0)  # 채널을 3번 반복하여 (3, H, W)로 확장
        # ToTensor 변환 적용
        image_tensor = torch.tensor(image_np, dtype=torch.float32)

        # (H, W, 3) -> (3, H, W)로 변환 필요 시 permute 사용
        if image_tensor.shape[0] != 3:
            image_tensor = image_tensor.permute(2, 0, 1)


        return image_tensor, label


In [ ]:
# EfficientNet-B0 모델을 위한 커스텀 클래스
class EfficientNetB0Model(nn.Module):
    def __init__(self, num_classes=4):
        super(EfficientNetB0Model, self).__init__()
        # 사전 학습된 EfficientNet-B0 모델 로드
        self.model = models.efficientnet_b0(pretrained=True)

        # 마지막 분류 레이어를 수정하여 num_classes에 맞게 설정
        in_features = self.model.classifier[1].in_features

        self.model.classifier = nn.Sequential(
            nn.Dropout(p=0.5),  # Dropout 추가 (과적합 방지)
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# 모델 초기화 및 설정
num_classes = 4  # 예측할 클래스 수
efficientnet_model = EfficientNetB0Model(num_classes=num_classes)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet_model = efficientnet_model.to(device)

In [ ]:
# 옵티마이저 및 학습 하이퍼파라미터 설정
learning_rate = 1e-6
optimizer = optim.Adam(efficientnet_model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# 데이터셋 경로 설정
train_data_path = '/content/drive/MyDrive/Medical_CV/3/전처리_데이터/train'
val_data_path = '/content/drive/MyDrive/Medical_CV/3/전처리_데이터/val'


train_dataset = CustomDataset(root_dir=train_data_path)
val_dataset = CustomDataset(root_dir=val_data_path)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 옵티마이저 Test

In [ ]:
# EfficientNet 모델 로드
model = models.efficientnet_b0(pretrained=True)

# 마지막 레이어를 클래스를 4개로 분류하도록 수정
num_classes = 4
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# 손실 함수 정의
criterion = torch.nn.CrossEntropyLoss()

# 옵티마이저 정의
optimizers = {
    "SGD": optim.SGD(model.parameters(), lr=0.001, momentum=0.9),
    "Adam": optim.Adam(model.parameters(), lr=0.001),
    "RAdam": optim.RAdam(model.parameters(), lr=0.001)
}

# 옵티마이저 비교 함수
def train_and_evaluate(optimizer, model, train_loader, val_loader, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"[{optimizer.__class__.__name__}] Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # 간단한 평가 (검증 세트에 대한 정확도)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"[{optimizer.__class__.__name__}] Validation Accuracy: {accuracy:.2f}%")

# 각 옵티마이저별로 학습 및 평가 수행
for opt_name, optimizer in optimizers.items():
    print(f"Training with {opt_name} optimizer")
    train_and_evaluate(optimizer, model, train_loader, val_loader, criterion)


Training with SGD optimizer
[SGD] Epoch [1/5], Loss: 0.7025
[SGD] Epoch [2/5], Loss: 0.2245
[SGD] Epoch [3/5], Loss: 0.1596
[SGD] Epoch [4/5], Loss: 0.1356
[SGD] Epoch [5/5], Loss: 0.1190
[SGD] Validation Accuracy: 89.87%
Training with Adam optimizer
[Adam] Epoch [1/5], Loss: 0.1886
[Adam] Epoch [2/5], Loss: 0.1288
[Adam] Epoch [3/5], Loss: 0.1166
[Adam] Epoch [4/5], Loss: 0.0920
[Adam] Epoch [5/5], Loss: 0.0731
[Adam] Validation Accuracy: 85.15%
Training with RAdam optimizer
[RAdam] Epoch [1/5], Loss: 0.0429
[RAdam] Epoch [2/5], Loss: 0.0391


# 좋은 결과를 보여주는 옵티마이저로 모델 학습 진행

In [ ]:
# 학습 함수 정의
def train_model(model, optimizer, criterion, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # 에포크마다 평균 손실 계산 및 출력
        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_train_loss:.4f}")

        # 검증 단계
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # 정확도 출력
        val_accuracy = 100 * correct / total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

# 모델 학습
train_model(efficientnet_model, optimizer, criterion, train_loader, val_loader, num_epochs=15)

Epoch [1/10], Loss: 1.2215
Validation Accuracy: 71.64%
Epoch [2/10], Loss: 0.8429
Validation Accuracy: 85.79%
Epoch [3/10], Loss: 0.5732
Validation Accuracy: 89.10%
Epoch [4/10], Loss: 0.3972
Validation Accuracy: 88.78%
Epoch [5/10], Loss: 0.2834
Validation Accuracy: 92.03%
Epoch [6/10], Loss: 0.2257
Validation Accuracy: 87.76%
Epoch [7/10], Loss: 0.1882
Validation Accuracy: 90.57%
Epoch [8/10], Loss: 0.1628
Validation Accuracy: 91.20%
Epoch [9/10], Loss: 0.1473
Validation Accuracy: 93.44%
Epoch [10/10], Loss: 0.1305
Validation Accuracy: 91.84%


In [ ]:
save_dir = '/content/drive/MyDrive/Medical_CV/models'
os.makedirs(save_dir, exist_ok=True)  # 경로가 없으면 생성

# 저장 파일 이름 설정
save_path = os.path.join(save_dir, 'efficientnetb0_weights.pth')

In [ ]:
torch.save(efficientnet_model.state_dict(), save_path)